# Results for decision tree classifier

In [ ]:
"""Results for decision tree classifier

"""

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler

# import data
df = pd.read_csv('data/SCADA_downtime_merged.csv', skip_blank_lines=True)

# list of turbines to plot
list1 = list(df['turbine_id'].unique())
# sort turbines in ascending order
list1 = sorted(list1, key=int)
# list of categories
list2 = list(df['TurbineCategory_id'].unique())
# remove NaN from list
list2 = [g for g in list2 if g >= 0]
# sort categories in ascending order
list2 = sorted(list2, key=int)
# categories to remove
list2 = [m for m in list2 if m not in (1, 12, 13, 14, 15, 17, 21, 22)]
list4 = list(range(0, 14))
list5 = list(zip(list4, list2))

# filter only data for turbine x
for x in list1:
    dfx = df[(df['turbine_id'] == x)].copy()
    # copying fault to new column (mins) (fault when turbine category id is y)
    for y in list2:
        def ff(c):
            if c['TurbineCategory_id'] == y:
                return 0
            else:
                return 1
        dfx['mins'] = dfx.apply(ff, axis=1)

        # sort values by timestamp in descending order
        dfx = dfx.sort_values(by='timestamp', ascending=False)
        # reset index
        dfx.reset_index(drop=True, inplace=True)

        # assigning value to first cell if it's not 0
        if dfx.loc[0, 'mins'] == 0:
            dfx.set_value(0, 'mins', 0)
        else:
            dfx.set_value(0, 'mins', 999999999)

        # using previous value's row to evaluate time
        for i, e in enumerate(dfx['mins']):
            if e == 1:
                dfx.at[i, 'mins'] = dfx.at[i-1, 'mins'] + 10

        # sort in ascending order
        dfx = dfx.sort_values(by='timestamp')
        # reset index
        dfx.reset_index(drop=True, inplace=True)
        # convert to hours, then round to nearest hour
        dfx['hours'] = dfx['mins'].astype(np.int64)
        dfx['hours'] = dfx['hours']/60
        dfx['hours'] = round(dfx['hours']).astype(np.int64)

        # > 48 hours - label as normal (9999)
        def f11(c):
            if c['hours'] > 48:
                return 9999
            else:
                return c['hours']
        dfx['hours'] = dfx.apply(f11, axis=1)

        # filter out curtailment - curtailed when turbine is pitching outside
        # 0 deg <= normal <= 3.5 deg
        def f22(c):
            if 0 <= c['pitch'] <= 3.5 or c['hours'] != 9999 or (
                    (c['pitch'] > 3.5 or c['pitch'] < 0) and (
                        c['ap_av'] <= (.1 * dfx['ap_av'].max())
                        or c['ap_av'] >= (.9*dfx['ap_av'].max()))):
                return 'normal'
            else:
                return 'curtailed'
        dfx['curtailment'] = dfx.apply(f22, axis=1)

        # filter unusual readings, i.e., for normal operation, power <= 0
        # in operating wind speeds, power > 100...
        # before cut-in, runtime < 600 and other downtime categories
        def f3(c):
            if c['hours'] == 9999 and ((
                        3 < c['ws_av'] < 25 and (
                            c['ap_av'] <= 0 or
                            c['runtime'] < 600 or
                            c['EnvironmentalCategory_id'] > 1 or
                            c['GridCategory_id'] > 1 or
                            c['InfrastructureCategory_id'] > 1 or
                            c['AvailabilityCategory_id'] == 2 or
                            12 <= c['TurbineCategory_id'] <= 15 or
                            21 <= c['TurbineCategory_id'] <= 22)) or
                    (c['ws_av'] < 3 and c['ap_av'] > 100)):
                return 'unusual'
            else:
                return 'normal'
        dfx['unusual'] = dfx.apply(f3, axis=1)

        # round to 6 hour intervals
        def f4(c):
            if c['hours'] == 0:
                return 10
            elif 1 <= c['hours'] <= 6:
                return 11
            elif 7 <= c['hours'] <= 12:
                return 12
            elif 13 <= c['hours'] <= 18:
                return 13
            elif 19 <= c['hours'] <= 24:
                return 14
            elif 25 <= c['hours'] <= 30:
                return 15
            elif 31 <= c['hours'] <= 36:
                return 16
            elif 37 <= c['hours'] <= 42:
                return 17
            elif 43 <= c['hours'] <= 48:
                return 18
            else:
                # normal
                return 19
        dfx['hours6'] = dfx.apply(f4, axis=1)

        # change label for unusual and curtailed data (20)
        def f5(c):
            if c['unusual'] == 'unusual' or c['curtailment'] == 'curtailed':
                return 20
            else:
                return c['hours6']
        dfx['hours_%s' % y] = dfx.apply(f5, axis=1)

        # drop unnecessary columns
        dfx = dfx.drop('hours6', axis=1)
        dfx = dfx.drop('hours', axis=1)
        dfx = dfx.drop('mins', axis=1)
        dfx = dfx.drop('curtailment', axis=1)
        dfx = dfx.drop('unusual', axis=1)

    # separate features from classes for classification
    features = [
        'ap_av', 'ws_av', 'wd_av', 'pitch', 'ap_max', 'ap_dev',
        'reactive_power', 'rs_av', 'gen_sp', 'nac_pos']
    classes = [col for col in dfx.columns if 'hours' in col]
    # list of columns to copy into new df
    list6 = features + classes
    df2 = dfx[list6].copy()
    # drop NaNs
    df2 = df2.dropna()
    X = df2[features]
    # normalise features to values b/w 0 and 1
    X = preprocessing.normalize(X)
    Y = df2[classes]
    # convert from pd dataframe to np array
    Y = Y.as_matrix()
    # cross validation using time series split
    tscv = TimeSeriesSplit(n_splits=5)

    dt = DecisionTreeClassifier(criterion='entropy')
    for (m, n) in list5:
        Ym = Y[:, m]
        # looping for each cross validation fold
        for train_index, test_index in tscv.split(X):
            # split train and test sets
            X_train, X_test = X[train_index], X[test_index]
            Y_train, Y_test = Ym[train_index], Ym[test_index]
            if len(set(Y_train)) > 1:
                ros = RandomOverSampler()
                Xt, Yt = ros.fit_sample(X_train, Y_train)
            else:
                Xt, Yt = X_train, Y_train
            # fit to classifier and predict
            dt1 = dt.fit(Xt, Yt)
            Yp = dt1.predict(X_test)
            print(
                'Classification report for turbine %s, turbine category %s'
                % (x, n))
            print(classification_report(Y_test, Yp, digits=6))
        print('-------------------------------------------------------------')

Classification report for turbine 1, turbine category 2.0
             precision    recall  f1-score   support

         19   0.989971  0.986451  0.988208     16311
         20   0.799456  0.843870  0.821062      1044

avg / total   0.978511  0.977874  0.978153     17355

Classification report for turbine 1, turbine category 2.0
             precision    recall  f1-score   support

         19   0.979488  0.991640  0.985526     15072
         20   0.939885  0.862900  0.899749      2283

avg / total   0.974278  0.974705  0.974243     17355

Classification report for turbine 1, turbine category 2.0
             precision    recall  f1-score   support

         19   0.948688  0.914576  0.931320      9845
         20   0.893057  0.935153  0.913620      7510

avg / total   0.924615  0.923480  0.923661     17355

Classification report for turbine 1, turbine category 2.0
             precision    recall  f1-score   support

         19   0.987159  0.990721  0.988937     13579
         20   0.

C:\Program Files\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Classification report for turbine 1, turbine category 3.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000       249
         11   0.000000  0.000000  0.000000       192
         12   0.000000  0.000000  0.000000       180
         13   0.000000  0.000000  0.000000       180
         14   0.000000  0.000000  0.000000       180
         15   0.000000  0.000000  0.000000       155
         16   0.000000  0.000000  0.000000       144
         17   0.000000  0.000000  0.000000       144
         18   0.000000  0.000000  0.000000       144
         19   0.931313  0.971739  0.951096     14260
         20   0.621185  0.906352  0.737150      1527

avg / total   0.819883  0.878191  0.846342     17355

Classification report for turbine 1, turbine category 3.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000         0
         11   0.000000  0.000000  0.000000         0
         12   0.000000  0.000000

C:\Program Files\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Classification report for turbine 1, turbine category 3.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000        84
         11   0.008772  0.027778  0.013333        36
         12   0.000000  0.000000  0.000000        36
         13   0.030303  0.055556  0.039216        36
         14   0.000000  0.000000  0.000000        36
         15   0.000000  0.000000  0.000000        36
         16   0.000000  0.000000  0.000000        36
         17   0.000000  0.000000  0.000000        36
         18   0.000000  0.000000  0.000000        36
         19   0.967791  0.965170  0.966478     13293
         20   0.941455  0.915176  0.928130      3690

avg / total   0.941528  0.934025  0.937717     17355

Classification report for turbine 1, turbine category 3.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000        17
         11   0.000000  0.000000  0.000000        72
         12   0.000000  0.000000

...

Classification report for turbine 25, turbine category 4.0
             precision    recall  f1-score   support

         19   0.991534  0.989894  0.990713     16327
         20   0.803571  0.830258  0.816697       813

avg / total   0.982618  0.982322  0.982459     17140

Classification report for turbine 25, turbine category 4.0
             precision    recall  f1-score   support

         19   0.993168  0.991287  0.992227     15839
         20   0.896319  0.916987  0.906535      1301

avg / total   0.985817  0.985648  0.985723     17140

Classification report for turbine 25, turbine category 4.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000        14
         11   0.000000  0.000000  0.000000        28
         12   0.000000  0.000000  0.000000        30
         13   0.000000  0.000000  0.000000        29
         14   0.000000  0.000000  0.000000        32
         15   0.000000  0.000000  0.000000        31
         16   0.00000

Classification report for turbine 25, turbine category 6.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000        47
         11   0.000000  0.000000  0.000000        56
         12   0.000000  0.000000  0.000000        53
         13   0.000000  0.000000  0.000000        64
         14   0.000000  0.000000  0.000000        48
         15   0.000000  0.000000  0.000000        56
         16   0.000000  0.000000  0.000000        57
         17   0.000000  0.000000  0.000000        55
         18   0.000000  0.000000  0.000000        61
         19   0.959706  0.990635  0.974925     14522
         20   0.933940  0.919849  0.926841      2121

avg / total   0.928690  0.953151  0.940706     17140

Classification report for turbine 25, turbine category 6.0
             precision    recall  f1-score   support

         11   0.000000  0.000000  0.000000         0
         12   0.000000  0.000000  0.000000         0
         13   0.000000  0.0000

Classification report for turbine 25, turbine category 9.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000         0
         11   0.000000  0.000000  0.000000         0
         12   0.000000  0.000000  0.000000         0
         13   0.000000  0.000000  0.000000         0
         14   0.000000  0.000000  0.000000         0
         15   0.000000  0.000000  0.000000         0
         16   0.000000  0.000000  0.000000         0
         17   0.000000  0.000000  0.000000         0
         18   0.000000  0.000000  0.000000         0
         19   0.989668  0.973847  0.981694     16327
         20   0.785104  0.790898  0.787990       813

avg / total   0.979965  0.965169  0.972506     17140

Classification report for turbine 25, turbine category 9.0
             precision    recall  f1-score   support

         11   0.000000  0.000000  0.000000         0
         12   0.000000  0.000000  0.000000         0
         13   0.000000  0.0000

Classification report for turbine 25, turbine category 11.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000         0
         11   0.000000  0.000000  0.000000         0
         12   0.000000  0.000000  0.000000         0
         13   0.000000  0.000000  0.000000         0
         14   0.000000  0.000000  0.000000         0
         15   0.000000  0.000000  0.000000         0
         16   0.000000  0.000000  0.000000         0
         17   0.000000  0.000000  0.000000         0
         18   0.000000  0.000000  0.000000         0
         19   0.993610  0.971842  0.982605     15839
         20   0.914122  0.736357  0.815666      1301

avg / total   0.987576  0.953967  0.969934     17140

Classification report for turbine 25, turbine category 11.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000         9
         11   0.000000  0.000000  0.000000        32
         12   0.000000  0.00

Classification report for turbine 25, turbine category 18.0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000         6
         11   0.012987  0.037037  0.019231        27
         12   0.000000  0.000000  0.000000        28
         13   0.000000  0.000000  0.000000        31
         14   0.000000  0.000000  0.000000        30
         15   0.000000  0.000000  0.000000        30
         16   0.000000  0.000000  0.000000        29
         17   0.000000  0.000000  0.000000        31
         18   0.000000  0.000000  0.000000        26
         19   0.918105  0.923670  0.920879     14516
         20   0.939370  0.500000  0.652626      2386

avg / total   0.908337  0.851925  0.870780     17140

Classification report for turbine 25, turbine category 18.0
             precision    recall  f1-score   support

         10   0.153846  0.142857  0.148148        14
         11   0.009709  0.017857  0.012579        56
         12   0.000000  0.00